In [1]:
# Packages
import os #directories
import pandas as pd # dataframe
import re # regular expressions

from bs4 import BeautifulSoup # webscrape
import urllib # webscrape
import requests # webscrape
from urllib.request import urlopen # webscrape
import csv # csv?

filedir = '/Users/Ryan/Desktop/Discord/data'

In [2]:
# set up
url = 'https://animalcrossing.fandom.com/wiki/Fish_(New_Horizons)#Northern%20Hemisphere'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')

In [3]:
# match for critter table
all_matches = soup.find("table",{"class":"sortable"})

In [4]:
# removes html tags
vinfo = BeautifulSoup(str(all_matches)).get_text()

# Lazy edit + split info by line into a list
fish = vinfo[114:].splitlines()
fish

['Bitterling',
 ' ',
 '  900',
 '  River',
 '  1',
 ' All day',
 ' ✓',
 ' ✓',
 ' ✓',
 ' -',
 ' -',
 ' -',
 ' -',
 ' -',
 ' -',
 ' -',
 ' ✓',
 ' ✓',
 '',
 '',
 ' Pale chub',
 ' ',
 '  200',
 '  River',
 '  1',
 ' 9 AM - 4 PM',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 '',
 '',
 ' Crucian carp',
 ' ',
 '  160',
 '  River',
 '  2',
 ' All day',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 '',
 '',
 ' Dace',
 ' ',
 '  240',
 '  River',
 '  3',
 ' 4 PM - 9 AM',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 '',
 '',
 ' Carp',
 ' ',
 '  300',
 '  Pond',
 '  4',
 ' All day',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 '',
 '',
 ' Koi',
 ' ',
 '  4000',
 '  Pond',
 '  4',
 ' 4 PM - 9 AM',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 '',
 '',
 ' Goldfish',
 ' ',
 '  1300',
 '  Pond',
 '  1',
 ' A

In [5]:
# data cleaning
# Remove trailing white space in each item
fish = [x.strip() for x in fish]

# remove blank items in bugs list
fish = [x for x in fish if x]

In [6]:
# Find all http links for pngs and return a list
pngurl = re.findall(r'https?://[^\s<>"]+|www\.[^\s<>"]+', str(all_matches))

# items were in tuple so combine them
pngurl = [''.join(tups) for tups in pngurl] 

# There are 3 png per insect. Take the first one of every bug
pngurl = pngurl[::3]

In [7]:
# Empty List to append to
rarity = []

# Prepare list of hyperlinks to iterate over
redirect = re.findall(r"/wiki/\w+[-]?[\(\][\w+]+[\)]?",str(all_matches))

# Go into each critter link page and find the rarity

root = 'https://animalcrossing.fandom.com/'
for x in redirect:
    goto = root + x
    #print(goto)
    response = requests.get(goto)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    
    all_matches = soup.find("div",{"data-source":"rarity"}) # find the rarity table at each critter page
    string = BeautifulSoup(str(all_matches)).get_text()
    rarity.append(string.splitlines()[2])

In [8]:
names = fish[::17]
price = fish[1::17]
location = fish[2::17]
shadow = fish[3::17]
time = fish[4::17]
jan = fish[5::17]
feb = fish[6::17]
mar = fish[7::17]
apr = fish[8::17]
may = fish[9::17]
jun = fish[10::17]
jul = fish[11::17]
aug = fish[12::17]
sep = fish[13::17]
oc = fish[14::17]
nov = fish[15::17]
dec = fish[16::17]

In [9]:
# Create Dictionary
dic = {'Name': names,
       'Price': price,
       'Location': location,
       'Shadow': shadow,
       'Time':time,
       'Rarity': rarity,
       'Jan': jan,
       'Feb': feb,
       'Mar': mar,
       'Apr': apr,
       'May': may,
       'Jun': jun,
       'Jul': jul,
       'Aug': aug,
       'Sep': sep,
       'Oct': oc,
       'Nov':nov,
       'Dec':dec,
       'pngurl':pngurl
      }

df=pd.DataFrame.from_dict(dic)
# searchable term
df['search'] = df.Name.apply(lambda x: (x.replace(" ", "")).lower())

In [10]:
# Replaces checks and minus with boolean

def conv(x):
    if x == '✓':
        return True
    elif x == '-':
        return False

# Function to clean Rarity column
def clean(x):
    a = re.findall(r'^[^\(]+',x)
    a = ' '.join(a)
    a = (a.strip()).lower()
    return a

In [11]:
# Fix
df.Jan = df.Jan.apply(lambda x: conv(x))
df.Feb = df.Feb.apply(lambda x: conv(x))
df.Mar = df.Mar.apply(lambda x: conv(x))
df.Apr = df.Apr.apply(lambda x: conv(x))
df.May = df.May.apply(lambda x: conv(x))
df.Jun = df.Jun.apply(lambda x: conv(x))
df.Jul = df.Jul.apply(lambda x: conv(x))
df.Aug = df.Aug.apply(lambda x: conv(x))
df.Sep = df.Sep.apply(lambda x: conv(x))
df.Oct = df.Oct.apply(lambda x: conv(x))
df.Nov = df.Nov.apply(lambda x: conv(x))
df.Dec = df.Dec.apply(lambda x: conv(x))
df.Rarity = df.Rarity.apply(lambda x: clean(x))

In [12]:
df

,Name,Price,Location,Shadow,Time,Rarity,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,pngurl,search
0,Bitterling,900,River,1,All day,common,True,True,True,False,False,False,False,False,False,False,True,True,https://vignette.wikia.nocookie.net/animalcros...,bitterling
1,Pale chub,200,River,1,9 AM - 4 PM,uncommon,True,True,True,True,True,True,True,True,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,palechub
2,Crucian carp,160,River,2,All day,fairly common,True,True,True,True,True,True,True,True,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,cruciancarp
3,Dace,240,River,3,4 PM - 9 AM,fairly common,True,True,True,True,True,True,True,True,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,dace
4,Carp,300,Pond,4,All day,fairly common,True,True,True,True,True,True,True,True,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,carp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Suckerfish,1500,Sea,4 (Fin),All day,uncommon,False,False,False,False,False,True,True,True,True,False,False,False,https://vignette.wikia.nocookie.net/animalcros...,suckerfish
76,Football fish,2500,Sea,4,4 PM - 9 AM,uncommon,True,True,True,False,False,False,False,False,False,False,True,True,https://vignette.wikia.nocookie.net/animalcros...,footballfish
77,Oarfish,9000,Sea,6,All day,scarce,True,True,True,True,True,False,False,False,False,False,False,True,https://vignette.wikia.nocookie.net/animalcros...,oarfish
78,Barreleye,15000,Sea,2,9 PM - 4 AM,scarce,True,True,True,True,True,True,True,True,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,barreleye


In [13]:
# Write to csv
df.to_csv(filedir + '/fish_data.csv', index = None, header = True)